In [4]:
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from gensim.models import Word2Vec, KeyedVectors


import nltk
import string
import gensim
import os
import collections
import re
import string
import pickle


import numpy as np
import pandas as pd

In [5]:
#dv data
data = pd.read_csv('dvdata.csv', header = None, delimiter=',')
data.columns = ['Id', 'Text']

#nondv data
dt=pd.read_csv('notdv.csv', header = None, delimiter=',')
dt.columns = ['Id', 'Text']

Label=[]
for l in data.Id:
    Label.append(1)
data['Label']=Label

Label=[]
for l in dt.Id:
    Label.append(0)
dt['Label']=Label
        
dvdt = pd.concat([data, dt], axis=0, ignore_index = True)
pos = []
neg = []
for l in dvdt.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
dvdt['Pos']= pos
dvdt['Neg']= neg
dvdt.shape

(4149, 5)

# Pre processing

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

def lower_token(tokens): 
    return [w.lower() for w in tokens]

def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

dvdt['Text_Clean'] = dvdt['Text'].apply(lambda x: remove_punct(x))

#lemmatization
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in dvdt.Text_Clean]
lower_tokens = [lower_token(token) for token in tokens]

#stopword_removal
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

filtered_words = [remove_stop_words(sen) for sen in lower_tokens]
result = [' '.join(sen) for sen in filtered_words]
dvdt['Text_Final'] = result
dvdt['tokens'] = filtered_words

dvdt = dvdt[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]
dvdt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deepak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deepak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Text_Final,tokens,Label,Pos,Neg
0,hate everything,"[hate, everything]",1,1,0
1,foul deeds rise fireopal19 darkest dark crime ...,"[foul, deeds, rise, fireopal19, darkest, dark,...",1,1,0
2,ecobard ’ let pass everything make substantiat...,"[ecobard, ’, let, pass, everything, make, subs...",1,1,0
3,rulebc15 theprojecttv stop treating dv gendere...,"[rulebc15, theprojecttv, stop, treating, dv, g...",1,1,0
4,observer view domestic violence observer edito...,"[observer, view, domestic, violence, observer,...",1,1,0
...,...,...,...,...,...
4144,2️⃣0️⃣ vintage 🤝 ive teamed awesome folks subs...,"[2️⃣0️⃣, vintage, 🤝, ive, teamed, awesome, fol...",0,0,1
4145,adoptionsuk looks gorgeous digital artist happ...,"[adoptionsuk, looks, gorgeous, digital, artist...",0,0,1
4146,comes unicorn squad artwork turnipberry find h...,"[comes, unicorn, squad, artwork, turnipberry, ...",0,0,1
4147,youfine dropping soon mampm maxistarpasachy ar...,"[youfine, dropping, soon, mampm, maxistarpasac...",0,0,1


# Word Embedding

In [7]:
data_train, data_test = train_test_split(dvdt, test_size=0.20, random_state=42)

all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

49895 words total, with a vocabulary size of 15643
Max sentence length is 60
12833 words total, with a vocabulary size of 5954
Max sentence length is 44


In [8]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
    return list(embeddings)

word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=100000)

MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 15643 unique tokens.
(15644, 300)


# Model Building

In [9]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [10]:
label_names = ['Pos', 'Neg']
y_train = data_train[label_names].values

x_train = train_cnn_data
y_tr = y_train
print(x_train.shape, y_tr.shape)

model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

(3319, 50) (3319, 2)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      4693200     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 49, 200)      120200      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 48, 200)      180200      ['embedding[0][0]']              
                                                                         

In [11]:
num_epochs = 3
batch_size = 34

hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.2, shuffle=True, batch_size=batch_size)

Epoch 1/3
79/79 [==============================] - 8s 88ms/step - loss: 0.3764 - acc: 0.8328 - val_loss: 0.2239 - val_acc: 0.8991
Epoch 2/3
79/79 [==============================] - 7s 84ms/step - loss: 0.1437 - acc: 0.9390 - val_loss: 0.1616 - val_acc: 0.9352
Epoch 3/3
79/79 [==============================] - 7s 84ms/step - loss: 0.0800 - acc: 0.9699 - val_loss: 0.1803 - val_acc: 0.9367


# Prediction

In [12]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

labels = [1, 0]

prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
    
print(accuracy_score(data_test.Label.values, prediction_labels))
print(confusion_matrix(data_test.Label.values, prediction_labels))

1/1 [==============================] - 1s 584ms/step
0.9313253012048193
[[420  22]
 [ 35 353]]


In [13]:
inputQuestion = remove_punct(input('Enter a tweet : '))
test_sequence = tokenizer.texts_to_sequences([inputQuestion])
test_sen = pad_sequences(test_sequence, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(test_sen)
labels = [1, 0]
pred = labels[np.argmax(prediction)]
print('Prediction : ', pred)

Enter a tweet : husband hurts
Prediction :  1


In [14]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

print(MAX_SEQUENCE_LENGTH)
with open('tokenizer.pickle', 'wb') as token:
    pickle.dump(tokenizer, token)
with open('max_seq_len.txt', 'w') as maxseq:
    maxseq.write(str(MAX_SEQUENCE_LENGTH))

Saved model to disk
50


In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import model_from_json


import pickle


import numpy as np


def predict(tweet):
    
    # load tokenizer
    with open('tokenizer.pickle', 'rb') as token:
        tokenizer = pickle.load(token)
    with open('max_seq_len.txt', 'r') as maxseq:
        MAX_SEQUENCE_LENGTH = maxseq.read()
        print()
        MAX_SEQUENCE_LENGTH = int(MAX_SEQUENCE_LENGTH)
        
        
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    
    # load weights into new model
    model.load_weights("model.h5")
    
    test_sequence = tokenizer.texts_to_sequences([tweet])
    test_sen = pad_sequences(test_sequence, maxlen=MAX_SEQUENCE_LENGTH)
    prediction = model.predict(test_sen)
    labels = [1, 0]
    pred = labels[np.argmax(prediction)]
    
    # prediction
    if pred == 1:
        print('Yes')
    else:
        print('No')
        
    return result

if __name__ == '__main__':
    tweet = input('Enter tweet : ')
    predict(tweet)

Enter tweet : happy birthday

No
